# Plot spot prices - extended

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
from projdirs import paperdir
from pandas.plotting import register_matplotlib_converters

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

sep = ['\\','/'][sys.platform=='linux']

# choose the state and date here
font_size = 14

def f(x,y):
    Index = (datetime.datetime.combine(x, y))
    if Index in data.index:
        return np.array(data.loc[Index]['spot_price'])
    else:
        return np.array(data.loc[Index-pd.Timedelta('1h')]['spot_price'])

def ff(x):
    return x.hour*60+x.minute

# plt.close("all")
fig = plt.figure(figsize=(16,18))
fig.tight_layout()
fig.subplots_adjust(left=0.0, bottom=0, right=1, top=1, wspace=0, hspace=0.1)

i = 0
for Year in [2010, 2019, 2022]:
    for Region in ['SA','BGE']:
            i+=1
            if Region in NEM:
                price_spot=gd.load_rrp_cal_stamped(Year, Region)
                price_spot['year']=Year
                price_spot.spot_price = price_spot[['spot_price','year']].apply(
                    lambda x: gd.convert_curr(*x ),axis=1)
                price_spot = price_spot.drop('year', axis=1)
                
            elif Region in PJM:
                price_spot=gd.load_lmp_zone_year(Region, Year, lmp_type='rt').reset_index()
                price_spot.columns = [ 'date_time', 'spot_price']
                price_spot['region'] = Region
                
            data = price_spot.set_index('date_time')
            x1 = pd.date_range(data.index.date.min(),
                               data.index.date.max(), freq='1D').date #2D
            y1 = pd.date_range('00:00:00','23:00:00', freq='60min').time
      
            x2, y2 = np.meshgrid(x1,y1)
      
            g = np.vectorize(f)
            gg = np.vectorize(ff)
            z = g(x2,y2)
            
            z[z<-50]=-50
            z[z>300]=300

            ax = fig.add_subplot(320+i, projection='3d', proj_type = 'ortho')
            
            
            surf = ax.plot_surface(dates.date2num(x2), gg(y2), z, rstride=1, cstride = 1,
                           edgecolor='k', linewidth=0.2, cmap=cm.jet)
                        
            x_tick_range = pd.date_range(data.index.date.min(),
                       data.index.date.max(), freq='1D').date
            ax.set_xlim(x_tick_range.min(), x_tick_range.max())
            ax.xaxis.set_tick_params(rotation=90 )
            
            ax.set_zticklabels(ax.axes.get_zticks(), horizontalalignment= 'left')
            ax.zaxis.set_major_formatter(FormatStrFormatter('%.0f'))
            
            ax.axes.set_yticks(np.arange(0,25*60,240))
            ax.axes.set_ylim([0,1450])
            ytick_range = ax.axes.get_yticks()
            m = len(ax.axes.get_yticks())
            ylabels = pd.date_range(start = '2000-01-01', end = '2000-01-02',  periods=m)
            ylabels = [datetime.datetime.strftime(Label, '%H:%M') for Label in ylabels]
            xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            ax.set_yticklabels(ylabels,rotation=-90, va='top', ha='right')
            ax.set_xticklabels(xlabels, va='top', ha='right')
            
            ax.set_zlabel('Spot price [USD/MWh]', fontsize=font_size, labelpad=20)
            ax.axes.invert_xaxis()
            ax.xaxis.set_tick_params(labelsize=font_size,rotation=45, pad=0.1)
            ax.yaxis.set_tick_params(labelsize=font_size)
            ax.zaxis.set_tick_params(labelsize=font_size)
            ax.set_zlim([-50,300])
            cbar = fig.colorbar(surf, ax=ax, shrink=.5, pad=0.07)
            cbar.ax.tick_params(labelsize=font_size)
            ax.set_title('%s - %d' %(['SA','PJM'][Region=='BGE'],Year), {
                        'fontsize': font_size,
#                         'fontweight' : '',
                        'verticalalignment': 'top'
                        })
                    
            ax.view_init(elev=30)
            ax.dist = 11

# plt.show()
# paperdir = 'C:\\Users\\Ahmad Mojiri\\OneDrive - Australian National University\Publications\paper_1\\'
# savedir = paperdir + 'Pictures%sRRP annual%sSurfplots%s'%(sep,sep,sep)
# plt.savefig(savedir + "RRP_surf_extended.png", format='png', dpi=300,  bbox_inches = "tight")
# plt.savefig(savedir + "RRP_surf_extended.svg", format='svg', bbox_inches = "tight")


# Plot price histograms

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from datetime import timedelta
import datetime
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pdb
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline

# choose the state and date here
PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

font_size = 14
Years = [2022]
Regions = ['BGE']

fig = plt.figure(figsize=(6,3))

for Year in Years:
    for Region in Regions:
        if Region in NEM:
            price_spot=gd.load_rrp_cal(Year, Region).reset_index()
            price_spot['year']=Year
            price_spot.spot_price = price_spot[['spot_price','year']].apply(
                lambda x: gd.convert_curr(*x ),axis=1)
            price_spot = price_spot.drop('year', axis=1)
            data = price_spot.set_index('date_time')

            

        elif Region in PJM:
            price_spot = gd.load_lmp_zone_year(Region, Year, lmp_type='rt').reset_index()
            price_spot.columns = [ 'date_time', 'spot_price']
            price_spot['region'] = Region

#         data = price_spot.set_index('date_time')
        
        RRPs = price_spot.spot_price.copy()
        RRPs[RRPs<-50]=-50
        RRPs[RRPs>300]=300
        
        
        plt.hist(RRPs,bins=50,
             range=[-50,300], edgecolor='k', color='orange', density=True)
        
        ax = plt.gca()
        plt.rc('ytick', labelsize=font_size)
        plt.rc('xtick', labelsize=font_size)
        plt.grid(which='both', axis='both')
        plt.xlim(-50, 300)
        plt.ylim(0, 0.07)
        ax.xaxis.set_tick_params(labelbottom=False)
        plt.tight_layout()
    # plt.show()
    # plt.savefig(
    # 'c:\\Nextcloud\\Thermal Battery Research\\Publications\\paper_1\\pictures\\RRP annual\\Histograms\\%s (Year %d)_Hist.png' %(State,year), dpi = 600)
    # plt.savefig(
    # 'c:\\Nextcloud\\Thermal Battery Research\\Publications\\paper_1\\pictures\\RRP annual\\Histograms\\%s (Year %d)_Hist.eps' %(State,year))
    

# Plot the spot prices (untruncated)

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import datetime
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm, dates
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import package.get_NEM_data as gd
from matplotlib.ticker import FormatStrFormatter
from projdirs import paperdir
from pandas.plotting import register_matplotlib_converters

PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

sep = ['\\','/'][sys.platform=='linux']

# choose the state and date here
font_size = 14

def f(x,y):
    Index = (datetime.datetime.combine(x, y))
    if Index in data.index:
        return np.array(data.loc[Index]['spot_price'])
    else:
        return np.array(data.loc[Index-pd.Timedelta('1h')]['spot_price'])

def ff(x):
    return x.hour*60+x.minute

# plt.close("all")
fig = plt.figure(figsize=(16,12))
fig.tight_layout()
fig.subplots_adjust(left=0.0, bottom=0, right=1, top=1, wspace=0.05, hspace=0.1)

i = 0
for Region in ['SA', 'BGE']:
      for Year in [2010,2019]:
            i+=1
            if Region in NEM:
                price_spot=gd.load_rrp_cal_stamped(Year, Region)
                price_spot['year']=Year
                price_spot.spot_price = price_spot[['spot_price','year']].apply(
                    lambda x: gd.convert_curr(*x ),axis=1)
                price_spot = price_spot.drop('year', axis=1)
                
            elif Region in PJM:
                price_spot=gd.load_lmp_zone_year(Region, Year, lmp_type='rt').reset_index()
                price_spot.columns = [ 'date_time', 'spot_price']
                price_spot['region'] = Region
                
            data = price_spot.set_index('date_time')
            x1 = pd.date_range(data.index.date.min(),
                               data.index.date.max(), freq='1D').date #2D
            y1 = pd.date_range('00:00:00','23:00:00', freq='60min').time
      
            x2, y2 = np.meshgrid(x1,y1)
      
            g = np.vectorize(f)
            gg = np.vectorize(ff)
            z = g(x2,y2)
            
#             z[z<-50]=-50
#             z[z>300]=300

            ax = fig.add_subplot(220+i, projection='3d', proj_type = 'ortho')
            
            
            surf = ax.plot_surface(dates.date2num(x2), gg(y2), z, rstride=1, cstride = 1,
                           edgecolor='k', linewidth=0.2, cmap=cm.jet)
                        
            x_tick_range = pd.date_range(data.index.date.min(),
                       data.index.date.max(), freq='1D').date
            ax.set_xlim(x_tick_range.min(), x_tick_range.max())
            ax.xaxis.set_tick_params(rotation=90 )
            
            ax.set_zticklabels(ax.axes.get_zticks(), horizontalalignment= 'left')
            ax.zaxis.set_major_formatter(FormatStrFormatter('%.0f'))
            
            ax.axes.set_yticks(np.arange(0,25*60,240))
            ax.axes.set_ylim([0,1450])
            ytick_range = ax.axes.get_yticks()
            m = len(ax.axes.get_yticks())
            ylabels = pd.date_range(start = '2000-01-01', end = '2000-01-02',  periods=m)
            ylabels = [datetime.datetime.strftime(Label, '%H:%M') for Label in ylabels]
            xlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            ax.set_yticklabels(ylabels,rotation=-90, va='top', ha='right')
            ax.set_xticklabels(xlabels, va='top', ha='right')
            
            ax.set_zlabel('Spot price [$/MWh]', fontsize=font_size, labelpad=30)
            ax.axes.invert_xaxis()
            ax.xaxis.set_tick_params(labelsize=font_size,rotation=45, pad=0.1)
            ax.yaxis.set_tick_params(labelsize=font_size)
            ax.zaxis.set_tick_params(labelsize=font_size)
#             ax.set_zlim([-50,300])
            cbar = fig.colorbar(surf, ax=ax, shrink=0.5, pad=0.04)
            cbar.ax.tick_params(labelsize=font_size)
            ax.set_title('%s - %d' %(Region,Year), {
                        'fontsize': font_size,
#                         'fontweight' : '',
                        'verticalalignment': 'top'
                        })
                    
            ax.view_init(elev=30)
            ax.dist = 11

# plt.show()
paperdir = 'C:\\Users\\z3337922\\OneDrive - Australian National University\\Publications\\paper_1\\'
savedir = paperdir + 'Pictures%sRRP annual%sSurfplots%s'%(sep,sep,sep)
plt.savefig(savedir + "RRP_surf_untruncated.png", format='png', dpi=300,  bbox_inches = "tight")
plt.savefig(savedir + "RRP_surf_untruncated.svg", format='svg', bbox_inches = "tight")


# Plot the SV including the arbitrage and capacity payments¶

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
import package.sql_manager as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']

NEM = ['SA','VIC','NSW','QLD','TAS']
PJM = ['DPL', 'BGE']

regions = ['SA','BGE']
titles = ['SA','PJM']

RTEs =np.arange(90,20,-20)
averagings = ['annually', 'quarterly']
averaging = averagings[0]

Fig = make_subplots(rows=1, cols=2, subplot_titles=titles,
                    shared_yaxes=True, horizontal_spacing=0.05)
Colors = ['orange','violet','orangered','deepskyblue','gold','mediumseagreen',
         'magenta','mediumslateblue']

for R,Region in enumerate(regions):
    
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        data = sm.get_data('N/A', 'N/A', db)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.arbi_type=='DA+RT')&
                        (data.zone==Region)&
                        (data.pout_max==50)&
                        (data.sh==10)&
                        (data.rte==RTE)]
            Fig.add_trace(go.Bar(x=DATA.year,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
    elif Region in NEM:
        db = 'SV_NEM_cap_contract.db'
        data = sm.get_data('N/A', 'N/A', db)
        data['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.state=='SA')&
                        (data.Pout_max==50)&
                        (data.sh==10)&
                        (data.cap_sampling==averaging)&
                        (data.rte==RTE)]
            
            Fig.add_trace(go.Bar(x=DATA.date_time,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
# Fix the layout
for i in [1,2]:
    Fig.update_xaxes({
#                       'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['USD/MW-year',''][i==2],
                      'ticks':'outside',
                      'range':[0,360000],
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=0.9, y=1),
                  legend_title_text = '         RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
paperdir = 'C:\\Users\\Ahmad Mojiri\\OneDrive - Australian National University\Publications\\paper_1\\'
savedir = paperdir + 'Pictures%sStorage value vs RTE and capacity%s'%(sep,sep)
# Fig.write_image(savedir + 'SV_RTE_%s_Nature.svg'%(" ".join(regions)))

# Plot the SV including the arbitrage and capacity payments - extended¶

In [ ]:
import sys, os
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

from projdirs import paperdir
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
import package.sql_manager as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sep = ['\\','/'][sys.platform=='linux']

NEM = ['SA','VIC','NSW','QLD','TAS']
PJM = ['DPL', 'BGE']

regions = ['SA','BGE']
titles = ['SA','PJM']

RTEs =np.arange(90,20,-20)
averagings = ['annually', 'quarterly']
averaging = averagings[0]

Fig = make_subplots(rows=1, cols=2, subplot_titles=titles,
                    shared_yaxes=True, horizontal_spacing=0.05)
Colors = ['orange','violet','orangered','deepskyblue','gold','mediumseagreen',
         'magenta','mediumslateblue']

for R,Region in enumerate(regions):
    
    if Region in PJM:
        db = 'storage_value_arbi_capacity_PJM.db'
        data = sm.get_data('N/A', 'N/A', db)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.arbi_type=='DA+RT')&
                        (data.zone==Region)&
                        (data.pout_max==50)&
                        (data.sh==10)&
                        (data.rte==RTE)]
            Fig.add_trace(go.Bar(x=DATA.year,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
    elif Region in NEM:
        db = 'SV_NEM_cap_contract.db'
        data = sm.get_data('N/A', 'N/A', db)
        data['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
        for r,RTE in enumerate(RTEs):
            DATA = data[(data.state=='SA')&
                        (data.Pout_max==50)&
                        (data.sh==10)&
                        (data.cap_sampling==averaging)&
                        (data.rte==RTE)]
            
            Fig.add_trace(go.Bar(x=DATA.date_time,
                                 y=DATA.SV_total/(50),
                                 name='%d%%'%(RTEs[r]),
                                 marker_color=Colors[r],
                                 marker_line_color='black',
                                 width=0.5,
                                 legendgroup='%d'%(RTE),
                                 showlegend = [False,True][R==1]),
                                 row=1, col=R+1)
# Fix the layout
for i in [1,2]:
    Fig.update_xaxes({
#                       'title':'Calendar year',
                      'tickmode': 'linear',
                      'tick0': 0,
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['USD/MW-year',''][i==2],
                      'ticks':'outside',
                      'range':[0,400000],
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=0.85, y=1),
                  legend_title_text = '         RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')
Fig.show()
paperdir = 'C:\\Users\\Ahmad Mojiri\\OneDrive - Australian National University\Publications\\paper_1\\'
savedir = paperdir + 'Pictures%sStorage value vs RTE and capacity%s'%(sep,sep)
Fig.write_image(savedir + 'SV_RTE_%s_Nature_extended.svg'%(" ".join(regions)))

# Plot the IRR vs SH including the cap contracts
This version includes the variable and fixed O&M costs too

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)

print("Working directory is now %s" %curr_path)
import numpy as np
from numpy_financial import irr
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import package.get_NEM_data as gd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import pdb

sep = ['\\','/'][sys.platform=='linux']
Pout_max = 50 #MWe
Pin_max = 50 #MWe
life = 20
Year = 2015

paperdir = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\'

excel_file = paperdir + 'Data\CPE_CPP.xlsx'
systems = pd.read_excel(excel_file, 'SummaryII',header=0, usecols='A:U',nrows=11)

base_year = 2015
Period = [2016, 2017, 2018, 2019, 2020]

Region = 'SA'
db='SV_NEM_cap_contract.db'
data = sm.get_data('N/A','N/A',db)

db = 'storage_value_PF_with_Eout_SA.db'
e_out = sm.get_data('N/A','N/A',db)


Fig = make_subplots(rows=2, cols=2, shared_xaxes=False, vertical_spacing=0.05)
name_list = []
subplot_id = {'Li': (1,1),
              'NaS': (1,1),
              'PHS': (1,2),
              'LA': (1,1),
              'VRFB': (1,1),
              'ZnBr': (2,2),
              'HFC': (2,1),
              'HFC': (2,1),
              'HCT': (2,1),
              'HGT': (2,1),
              'CAES': (1,2),
              'HFC': (2,1),
              'PTES': (3,1),
              'MS': (2,1),
              'LAES': (3,1),
              'MS-SC':(2,1),
              'MS-EPP':(2,1)}
text_loc = {'purple':18,
            'deepskyblue': 8,
            'tomato': 11,
            'magenta': 14,
            'green': 4,
            'gray': 16,
            'brown': 12,
            'lightcoral':8,
            'black':10}

for i in systems.index[0:10]:
    IRR = pd.DataFrame(columns=['sh','irr_median','irr_mean','irr_std'])
    RTE = int(100*systems.loc[i,'Efficiency'])
    
    CPP_median = gd.time_value(systems.loc[i,'CPP_median'],systems.loc[i,'Year'],base_year,0.02) 
    CPP_low = gd.time_value(systems.loc[i,'CPP_low'],systems.loc[i,'Year'],base_year,0.02)
    CPP_high = gd.time_value(systems.loc[i,'CPP_high'],systems.loc[i,'Year'],base_year,0.02)
    
    BOP_median = gd.time_value(systems.loc[i,'BOP_median'],systems.loc[i,'Year'],base_year,0.02)
    BOP_low = gd.time_value(systems.loc[i,'BOP_low'],systems.loc[i,'Year'],base_year,0.02)
    BOP_high = gd.time_value(systems.loc[i,'BOP_high'],systems.loc[i,'Year'],base_year,0.02)
    
    CPE_median = gd.time_value(systems.loc[i,'CPE_median'],systems.loc[i,'Year'],base_year,0.02)
    CPE_low = gd.time_value(systems.loc[i,'CPE_low'],systems.loc[i,'Year'],base_year,0.02)
    CPE_high = gd.time_value(systems.loc[i,'CPE_high'],systems.loc[i,'Year'],base_year,0.02)
    
    CC_median = gd.time_value(systems.loc[i,'C&C_median'],systems.loc[i,'Year'],base_year,0.02)
    CC_low = gd.time_value(systems.loc[i,'C&C_low'],systems.loc[i,'Year'],base_year,0.02)
    CC_high = gd.time_value(systems.loc[i,'C&C_high'],systems.loc[i,'Year'],base_year,0.02)
    
    fom = gd.time_value(systems.loc[i,'FOM'],systems.loc[i,'Year'],base_year,0.02) #$/kW-year
    vom = gd.time_value(systems.loc[i,'VOM'],systems.loc[i,'Year'],base_year,0.02) #$/kWh
    
    sv = data[(data.state==Region)&
              (data.loss==0)&
              (data.cap==14500)&
              (data.Pin_max==Pin_max)&
              (data.Pout_max==Pout_max)&
              (data.date_time.isin(Period) )&
              (data.cap_sampling=='annually')&
              (data.rte==RTE)].sort_values('date_time').copy()
    
    E_out = e_out[(e_out.state==Region)&
                  (e_out.loss==0)&
                  (e_out.cap==14500)&
                  (e_out.Pin_max==Pin_max)&
                  (e_out.Pout_max==Pout_max)&
                  (e_out.date_time.isin(Period) )&
                  (e_out.rte==RTE)].sort_values('date_time').copy()


    # Convert the NEM SV to usd
    sv['SV_total']= sv[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
    # convert to 2015 USD
    sv['SV_total'] = sv.apply(lambda x: gd.time_value(x['SV_total'],x['date_time'],base_year,0.02 ),axis=1)
    
    SV = sv.groupby('sh').mean().sort_values('sh').reset_index()
    E_OUT = E_out.groupby('sh').mean().sort_values('sh').reset_index()
    
    for s,SH in enumerate(SV.sh[0:20]):
        CAPEX = Pout_max*(CPP_median+BOP_median)*1e3+(CPE_median+CC_median)*Pout_max*SH*1e3
        FOM = fom*Pout_max*1e3
        VOM = vom*E_OUT[E_OUT.sh==SH]['Eout'].values[0]*1000
        rev = SV[SV.sh==SH]['SV_total'].values[0]
        cashflows = np.insert((rev-FOM-VOM)*np.ones(life),0,-CAPEX)
        IRR_median = ( round(irr(cashflows),4)  )
        
        n=10000
        sample = 1000
        
        CPPs = np.concatenate((
            np.array(random.sample(range(int(n*CPP_low),int(n*CPP_median)),sample))/n,
            np.array(random.sample(range(int(n*CPP_median),int(n*CPP_high)),sample))/n)
                             )
        
        CPEs = np.concatenate((
            np.array(random.sample(range(int(n*CPE_low),int(n*CPE_median)),sample))/n,
            np.array(random.sample(range(int(n*CPE_median),int(n*CPE_high)),sample))/n
                              )) 

        BOPs = np.concatenate((
            np.array(random.sample(range(int(n*BOP_low),int(n*BOP_median)),sample))/n,
            np.array(random.sample(range(int(n*BOP_median),int(n*BOP_high)),sample))/n,
                                ))
        CCs = np.concatenate((
            np.array(random.sample(range(int(n*CC_low),int(n*CC_median)),sample))/n,
            np.array(random.sample(range(int(n*CC_median),int(n*CC_high)),sample))/n
                            ))

        random.shuffle(CPPs)
        random.shuffle(BOPs)
        random.shuffle(CPEs)
        random.shuffle(CCs)

        IRR_list=[]
        for cpp,cpe in zip(CPPs+BOPs,CPEs+CCs):
            capex = Pout_max*cpp*1000+cpe*1000*Pout_max*SH
            rev = SV[SV.sh==SH]['SV_total'].values[0]
            cashflows = np.insert((rev-FOM-VOM)*np.ones(life),0,-capex)
            IRR_list.append ( round(irr(cashflows),4)  )

        IRR=IRR.append(dict(sh=SH,
                            irr_median=IRR_median,
                            irr_mean=np.mean(IRR_list),
                            irr_std=np.std(IRR_list)
                           ),ignore_index = True
                      )
        
    
    Name = systems.loc[i,'Technology']
    
    # add the curve
    Fig.add_trace(go.Scatter(x = IRR.sh, y=IRR.irr_median,
                             name = Name, legendgroup = Name, 
                             showlegend = [False,False][Name in name_list],
                             mode='lines',
                             line=dict(color=systems.loc[i,'Line_Color'], 
                                       dash=['dash','solid'][int(systems.loc[i,'Proven'])])
                             ),
                  row=subplot_id[Name][0],col = subplot_id[Name][1])
    
    # add the error bars
    Fig.add_trace(go.Scatter(x= IRR.sh[np.mod(i,2)::2],
                             y=IRR.irr_mean[np.mod(i,2)::2],
                             error_y=dict(type='data',
                                          thickness=1,
                                          array=IRR.irr_std,
                                          visible=True),
                             mode='markers',
                             marker=dict(color=systems.loc[i,'Line_Color'], size=1),
                             showlegend=False
                             ),
                  row=subplot_id[Name][0],col = subplot_id[Name][1])
    
#     t = text_loc[systems.loc[i,'Line_Color']]
    t = systems.loc[i,'Annotate']
    Fig.add_annotation(x=IRR.sh[t],y=IRR.irr_median[t], text=Name,
                       font=dict(size=12, color=systems.loc[i,'Line_Color']),
                       row=subplot_id[Name][0],col=subplot_id[Name][1], bgcolor="white")
    name_list.append(Name)

    
for Row in [1,2,3]:
    for Col in [1,2]:

        Fig.update_xaxes({'title':['SH [Hours]',''][(Row==1)&(Col==1)],
                          'tick0': 0.5,
                          'tickvals':np.insert(np.arange(2,20,2.0),0,0.5,axis=0), 
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True,
                          'showticklabels':[True,False][(Col==1)&(Row==1)]},row=Row, col=Col)
        Fig.update_yaxes({'title':['IRR',''][(Row==2) & (Col==2)],
                          'ticks':'outside',
                          'nticks':10,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True},row=Row, col=Col)

Text = ("<b>Li</b>: Lithium-ion battery (RTE=%1.1f)<br>"%(
        int(systems.set_index('Technology').loc['Li'].Efficiency*100)/100)+
        
        "<b>VRFB</b>: Vanadium Redox Flow Battery (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['VRFB'].Efficiency*100)/100)+

        "<b>NaS</b>: Sodium Sulfur battery (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['NaS'].Efficiency*100)/100)+

        "<b>LA</b>: Lead Acid battery (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['LA'].Efficiency*100)/100)+

        "<b>CAES</b>: Compressed Air Energy Storage (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['CAES'].Efficiency*100)/100)+

        "<b>PHS</b>: Pumped Hydro Storage (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['PHS'].Efficiency*100)/100)+

        "<b>MS</b>: Molten Salt with steam turbine (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['MS'].Efficiency*100)/100)+

        "<b>HGT</b>: underground Hydrogen + Gas Turbine (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['HGT'].Efficiency*100)/100)+

        "<b>HFC</b>: underground Hydrogen + Fuel Cell (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['HFC'].Efficiency*100)/100)+
        
        "<b>MS-SC</b>: Molten Salt + S.C. steam turbine (RTE=%1.1f)<br>"%(
            int(systems.set_index('Technology').loc['MS-SC'].Efficiency*100)/100)
        
#         "<b>MS-EPP</b>: Molten Salt in Power Plants (RTE=%1.1f)<br>"%(
#             int(systems.set_index('Technology').loc['MS-EPP'].Efficiency*100)/100)
       )

Fig.add_annotation(text=Text,
                   xref="paper", yref="paper",
                   align='left',
                   x=1, y=0.0, showarrow=False)

Fig.update_layout(plot_bgcolor='white',
                  width=850, height=500,
                  margin=dict(l=10, r=10, t=20, b=10))
    
savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)
Fig.write_image(savedir + 'IRR_SH_%s_%d-2020(with FOM) without retrofit_extended.svg'%(Region, Year+1))
Fig.show()

# Arbitrage net revenue as a function of foresight horizon

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)


from projdirs import datadir, paperdir
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import package.sql_manager as sm
import pdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sep = ['\\','/'][sys.platform=='linux']

regions = ['SA','BGE']
RTEs = [40,90]
SH = 10
Loss = 0
cap = 14500

PJM = ['PJM-RTO', 'MID-ATL', 'APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','VIC','NSW','QLD','TAS']
colors = ['orangered','lightcoral','goldenrod','gold','deepskyblue']
windows = [1,2,3,4]
Width=0.2
Opacity = 1
years = np.arange(2010,2021,1)
Fig = make_subplots(rows=2,cols=2,
                    subplot_titles=['%s (RTE=%d%%)'%(x,y) for x,y in zip(2*regions,np.repeat(RTEs,2))],
                    shared_xaxes=True,
                    shared_yaxes=True,
                    vertical_spacing=0.15,
                    horizontal_spacing=0.05)

for r,RTE in enumerate(RTEs):
    
    for R,Region in enumerate(regions):
        if Region in NEM:
            db = 'storage_value_PF_simple_%s.db'%(Region)
            data_pf = sm.get_data('N/A','N/A', db)
            
            SV_pf=data_pf[(data_pf.sh==10)&
                          (data_pf.rte==RTE)&
                          (data_pf.loss==0)&
                          (data_pf.cap==14500)&
                          (data_pf.Pout_max==50)&
                          (data_pf.Pin_max==50)].\
                  sort_values('date_time')[['date_time','SV']].set_index('date_time')

            db = 'storage_value_short_foresight_%s.db'%(Region)
            data_sf = sm.get_data('N/A','N/A', db)
            data_sf['date_time'] = data_sf['date_time'].astype(int)
            
            SV_sf= data_sf[(data_sf.sh==10)&
                           (data_sf.rte==RTE)&
                           (data_sf.loss==0)&
                           (data_sf.cap==14500)&
                           (data_sf.Time_gen=='00:00:00')].sort_values('date_time')
            for w,Window in enumerate(windows):
                SV_SF = SV_sf[SV_sf.window==Window][['date_time','SV']].\
                        sort_values('date_time').set_index('date_time')
                Fig.add_trace(go.Bar(x=SV_SF.index,
                                     y=SV_SF.SV/SV_pf.SV,
                                     name='%d-day'%(Window),
                                     marker_color=colors[w],
                                     marker_line_color='black',
                                     opacity=Opacity,
                                     width=Width,
                                     legendgroup='%d-day'%(Window),
                                     showlegend = [False,True][r==1]),
                                     row=r+1, col=R+1)
        elif Region in PJM:
            db = 'storage_value_PF_simple_PJM.db'
            data_pf = sm.get_data('N/A','N/A', db)
            SV_pf=data_pf[(data_pf.zone==Region)&
                          (data_pf.sh==10)&
                          (data_pf.rte==RTE)&
                          (data_pf.loss==0)&
                          (data_pf.Pout_max==50)&
                          (data_pf.Pin_max==50)].\
                     sort_values('date_time')[['date_time','SV']].set_index('date_time')

            db = 'storage_value_short_foresight_%s.db'%(Region)
            data_sf = sm.get_data('N/A','N/A', db)
            data_sf['date_time'] = data_sf['date_time'].astype(int)
            
            SV_sf= data_sf[(data_sf.sh==10)&
                           (data_sf.rte==RTE)&
                           (data_sf.loss==0)&
                           (data_sf.Time_gen=='00:00:00')].sort_values('date_time')
            
            for w,Window in enumerate(windows):
                SV_SF = SV_sf[SV_sf.window==Window][['date_time','SV']].\
                        sort_values('date_time').set_index('date_time')
                Fig.add_trace(go.Bar(x=SV_SF.index,
                                     y=SV_SF.SV/SV_pf.SV,
                                     name='%d-day'%(Window),
                                     marker_color=colors[w],
                                     marker_line_color='black',
                                     width=Width,
                                     opacity=Opacity,
                                     legendgroup='%d-day'%(Window),
                                     showlegend = False),
                                     row=r+1, col=R+1)

# Fix the layout
for i in [1,2]:
    for j in [1,2]:
        
        Fig.update_xaxes({'title':'',
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 1,
                          'ticks':'outside',
                          'tickangle':30,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':['ANR/ANR<sub>PF<sub>',''][j==2],
                          'ticks':'outside',
                           'dtick':0.1,
                          'range':[0.6,1.05],
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.1,
                              xanchor='center',
                              yanchor='bottom'),
                  legend_title_text = '',
                  barmode='group',
                  width=900, height=400,
                  margin=dict(l=10, r=10, t=20, b=10),
                 )

# paperdir = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\'
# savedir = paperdir + 'Pictures%sPerfect foresigth vs short foresight%s'%(sep,sep)

# Fig.write_image(savedir + 'SV_FS_%s_extended.png'%(" ".join(regions)))
Fig.show()    

# IRR vs CPE and CPP with cap contracts for SA

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)

curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import numpy_financial as npf
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
from matplotlib import container, cm
import itertools
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline
plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']
def er(x,y):
    return(y-av([x,y]))

CPE_range = np.array(list(
                        itertools.chain(np.arange(1,10,1),
                                        np.arange(10,110,2),#2
                                        np.arange(110,900,5),#5
                                        np.arange(900,1500,10)
                                       )
                         ) )# $/kWh

                           
CPP_range = np.array(list( 
                        itertools.chain(np.arange(10,200,2),#2
                                        np.arange(200,6000,20) )#20
                        ) ) # $/kW


IRR = np.zeros( (len(CPP_range), len(CPE_range)) )
RTEs = [40,50,60,70,80,90]
Pout_max = 50 #MWe
Pin_max = 50 #MWe
SH = 1
Years = [2016,2017,2018,2019,2020]
life = 20

excel_file = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\Data\CPE_CPP.xlsx'
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:O',nrows=37)

fig, Axes = plt.subplots(figsize=(16,18),nrows=3, ncols=2, gridspec_kw = {'wspace':0.05, 'hspace':0.1})
Ax=np.ravel(Axes)
font_size = 16


Region = 'SA'
db='SV_NEM_cap_contract.db'
data = sm.get_data('N/A','N/A',db)

db = 'storage_value_PF_with_Eout_SA.db'
e_out = sm.get_data('N/A','N/A',db)

SV = data[(data.state==Region)&
          (data.sh==SH)&
          (data.loss==0)&
          (data.cap==14500)&
          (data.Pin_max==Pin_max)&
          (data.Pout_max==Pout_max)&
          (data.date_time.isin(Years))].sort_values('date_time')#['SV']

E_out = e_out[(e_out.state==Region)&
              (e_out.loss==0)&
              (e_out.cap==14500)&
              (e_out.sh==SH)&
              (e_out.Pin_max==Pin_max)&
              (e_out.Pout_max==Pout_max)&
              (e_out.date_time.isin(Years))].sort_values('date_time').copy()



# Convert the NEM SV to usd
SV['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
SV['SV']= data[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
# convert to 2015 USD
SV['SV'] = SV.apply(lambda x: gd.time_value(x['SV'],x['date_time'],2015,0.02 ),axis=1)
SV['SV_total'] = SV.apply(lambda x: gd.time_value(x['SV_total'],x['date_time'],2015,0.02 ),axis=1)


for r,RTE in enumerate(RTEs):
    i=int(np.divide(r,2))+1
    j=int(np.mod(r,2))+1
    SV_rte = SV[SV['rte']==RTE]['SV_total']
    SV_mean = SV_rte.mean()
    E_rte = E_out[E_out.rte==RTE]['Eout']
    E_OUT = E_rte.mean()
    
    for cpe,CPE in enumerate(CPE_range):
        for cpp,CPP in enumerate(CPP_range):
            CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3
            cashflows = np.insert(SV_mean*np.ones(life),0,-CAPEX)
            IRR[cpp,cpe] = round(npf.irr(cashflows),8)

    X, Y = np.meshgrid(CPE_range, CPP_range)
    IRR[IRR>1.0]=1.01
    IRR[IRR<0]=-0.0001
    
#     L = [0,0.03,0.06,0.08,0.12,0.18,0.3, 0.4, 0.5, 0.6, 0.9, 1]
    L=[0, 0.03, 0.06, 0.08, 0.12, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8]
    
    CMAP = cm.get_cmap('CMRmap')#BuGn  Wistia GnBu  YlGn  Blues  GnBu gnuplot jet
    colors = CMAP(np.linspace(0.65, 1, num=len(L)+1))
    colors_rep = [x for x in colors.tolist() for i in range(2)]
    
    L_rep = [x for x in L for i in range(2)]
    L_rep[0]=0.0005
    L_rep[1]=0.0005
    L_rep[-2]=0.8
    L_rep[-1]=0.8
    L_rep = [0]+L_rep+[1]
    
    CMAP = LinearSegmentedColormap.from_list(
                'mycmap', [x for x in zip(L_rep,colors_rep)])
    
    im = Ax[r].contourf(X,Y, IRR, 1000, cmap=CMAP)
    Ax[r].set_yscale('log')
    Ax[r].set_xscale('log')
    
    im2 = Ax[r].contour(X,Y, IRR, colors='k',levels=L)
    clabels = Ax[r].clabel(im2, inline_spacing=5, colors = 'k', fmt = '$%2.2f$', fontsize=font_size-2)

    Ax[r].grid(which='both', axis='both')
    if r in [0,2,4]:
        Ax[r].set_ylabel(r'CPP [$\$$/kW]', fontsize=font_size)
    if r in [4,5]:
        Ax[r].set_xlabel(r'CPE [$\$$/kWh]', fontsize=font_size)
    if r in [1,3,5]:
        Ax[r].tick_params(labelleft=False)
    if r in [0,1,2,3]:
        Ax[r].tick_params(labelbottom=False)
    
    Systems = systems[(systems['Eff_low']*100<=RTE) & (systems['Eff_high']*100>=RTE)]
    for i in Systems.index:
        CPE_mean = Systems.loc[i,'CPE_mean']
        CPP_mean = Systems.loc[i,'CPP_mean']
        CAPEX = Pout_max*(CPP_mean)*1e3+(CPE_mean)*Pout_max*SH*1e3
        FOM = Systems.loc[i,'FOM']*Pout_max*1e3
        VOM = Systems.loc[i,'VOM']*E_OUT*1e3
        
        cashflows = np.insert((SV_mean-FOM-VOM)*np.ones(life),0,-CAPEX)
        IRR_median = ( round(npf.irr(cashflows),4)  )
        
        FOM_npv = npf.npv(IRR_median,np.insert(FOM*np.ones(life),0,0))
        VOM_npv = npf.npv(IRR_median,np.insert(VOM*np.ones(life),0,0))
        
        CPP_fom = FOM_npv/(Pout_max*1e3)
        CPE_vom = VOM_npv/(Pout_max*SH*1e3)
        
       
        Ax[r].errorbar(CPE_mean+CPE_vom, CPP_mean+CPP_fom,
                       xerr=np.array([[Systems.loc[i,'CPE_err-'],Systems.loc[i,'CPE_err+']]]).T, 
                       yerr=np.array([[Systems.loc[i,'CPP_err-'],Systems.loc[i,'CPP_err+']]]).T, 
                       marker=Systems.loc[i,'Marker'],
                       color=Systems.loc[i,'color'],
                       mec='b',ms=10, label = Systems.loc[i,'Type_ref']
                      ,ecolor='b', capsize=5,linestyle = 'None')
       
    Ax[r].text(0.87,0.93,'RTE=%d\%%'%(RTE),
              fontsize=font_size, transform=Ax[r].transAxes,
               ha='center', bbox=dict(facecolor='w', edgecolor='k'), zorder=10)

    Ax[r].set_ylim([CPP_range.min(),CPP_range.max()])
    Ax[r].set_xlim([CPE_range.min(),CPE_range.max()])
    handles, labels = Ax[r].get_legend_handles_labels()
    handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
    lg = Ax[r].legend(handles, labels,
                      loc='lower left', ncol=1,
                      fontsize=font_size-4,
                      edgecolor='k',
                      handletextpad=0.5,
                      framealpha=0.8)

import matplotlib.ticker as tick
# cbar.ax.yaxis.set_major_formatter(tick.FormatStrFormatter('%.1f'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sIRR_CPE_CPP%s'%(sep,sep)

plt.savefig(savedir + 'IRR_CPP_CPE_%d-2020 (SH=%d)_with_cap_ with retrofit_Nature_extended.png'%(2016,SH),dpi=600,  format='png',bbox_inches = "tight")
plt.savefig(savedir + 'IRR_CPP_CPE_%d-2020 (SH=%d)_with_cap_ with retrofit_Nature_extended.svg'%(2016,SH),
            bbox_inches = "tight")
plt.show()

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)

curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir
import package.sql_manager as sm
from datetime import timedelta, time
import package.storage_analysis as sa
import package.get_NEM_data as gd
from calendar import monthrange
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sep = ['\\','/'][sys.platform=='linux']


states = ['SA']
Loss = 0
RTEs=[40,90]
SHs=[10]
price_cap = 14500
Window = 1
years = np.arange(2010,2021,1)
Fig = make_subplots(subplot_titles=['RTE=%d%%'%(RTEs[0]), 'RTE=%d%%'%(RTEs[1])],
                    shared_yaxes=True,
                    rows=1, cols=2, horizontal_spacing=0.07)

colors = ['pink','tomato','red','gray']

for r,RTE in enumerate(RTEs):
    for SH in SHs:
        for s,State in enumerate(states):

            db = 'storage_value_PF_simple_%s.db'%(State)
            data = sm.get_data('N/A', 'N/A',db)
            SV_perf = data[(data.sh==10) &
                           (data.loss==0) &
                           (data.cap==14500) &
                           (data.rte==RTE) &
                           (data.Pin_max==50) &
                           (data.Pout_max==50)].sort_values('date_time', axis=0).set_index('date_time')


    # exact forecast
            db = 'storage_value_short_foresight_%s.db'%(State)
            data = sm.get_data('N/A', 'N/A',db)
            data['date_time']=data.date_time.astype(int)

            SV_exact = data[(data.sh==SH)&
                            (data.rte==RTE)&
                            (data.window==1)&
                            (data.Time_gen=='00:00:00')].sort_values('date_time', axis=0).set_index('date_time')
    # Naive forecast
            db = 'storage_value_naive_foresight_%s.db'%(State)
            data = sm.get_data('N/A', 'N/A',db)
            data['date_time']=data.date_time.astype(int)
            SV_naive = data[(data.sh==SH)&
                            (data.rte==RTE)&
                            (data.Time_gen=='00:00:00') ].sort_values('date_time', axis=0).set_index('date_time')


    # Daily forecast
            db= 'SV_CalDay_forecast_time_%s.db'%('SA')
            data = sm.get_data('N/A','N/A',db)
            data = data[(data.onset=='23:30:00')&
                        (data.forecast_time=='23:00:00')&
                        (data.rte==RTE) &
                        (data.sh==SH)]
            SV_fc = data[['region','date_time','obj_actual']]

            SV_fc = SV_fc.sort_values('date_time', axis=0).set_index('date_time')

    # Rolling forecast
            db = 'storage_value_rolling_forecast_%s.db'%(State)
            Storage_Rolling = sm.get_data('N/A','N/A',db)
            Storage_Rolling['date_time']=pd.to_datetime(Storage_Rolling.date_time)

            SV_rolling = pd.DataFrame(columns=['year','state','SV'])
            for Year in years:
                storage_rolling = Storage_Rolling[(Storage_Rolling.state==State) &
                                                  (Storage_Rolling.rte==RTE) &
                                                  (Storage_Rolling.sh==SH) &
                                                  (Storage_Rolling.date_time.dt.year==Year)]

                SV = (0.5*(storage_rolling.Pout-storage_rolling.Pin)*
                              storage_rolling.RRP).sum()

                SV_rolling = SV_rolling.append({'state':State,                                               
                                                'year':Year,                                               
                                                'SV':SV}, ignore_index=True)
            SV_rolling=SV_rolling.set_index('year')

            Fig.add_trace(go.Bar(x=SV_naive.index,
                                 y=SV_naive.SV/SV_perf.SV,
                                 marker_color=colors[0],
                                 marker_line_color='black',
                                 showlegend=[False, True][(r==1) & (SH==10)],
                                 name='%s forecast'%('Naive'),
#                                  offsetgroup = 'Naive'
                                 ), row=1, col=r+1)
            Fig.add_trace(go.Bar(x=SV_fc.index,
                                 y=SV_fc.obj_actual/SV_perf.SV,
                                 marker_color=colors[1],
                                 marker_line_color='black',
                                 showlegend=[False, True][(r==1) & (SH==10)],
                                 name='Predictive 24-hr forecast',
#                                  offsetgroup = '24-hr'
                                 ), row=1, col=r+1)
            Fig.add_trace(go.Bar(x=SV_rolling.index,
                                 y=SV_rolling.SV/SV_perf.SV,
                                 marker_color=colors[2],
                                 marker_line_color='black',
                                 showlegend=[False, True][(r==1) & (SH==10)],
                                 name='Predictive rolling forecast',
#                                  offsetgroup = 'Rolling'
                                 ), row=1, col=r+1)
            Fig.add_trace(go.Bar(x=SV_exact.index,
                                 y=SV_exact.SV/SV_perf.SV,
                                 marker_color=colors[3],
                                 marker_line_color='black',
                                 showlegend=[False, True][(r==1) & (SH==10)],
                                 name='One-day exact foresight',
#                                  offsetgroup = 'One_day'
                                 ), row=1, col=r+1)

for i in [1,2]:
    Fig.update_xaxes({'title':'',
                      'dtick': 1,
                      'ticks':'outside',
                      'tickangle':30,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)
    Fig.update_yaxes({'title':['ANR/ANR<sub>PF</sub>', ''][i==2],
                      'ticks':'outside',
                      'dtick': 0.2,
                      'showgrid':True,
                      'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=i)

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.1,
                              xanchor='center',
                              yanchor='bottom'),
                  width=900, height=350,
                  barmode='group',
                  margin=dict(l=10, r=5, t=20, b=10),
                  )

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1"
savedir = paperdir + '\Pictures%sForecast comparison%s'%(sep,sep) 

Fig.write_image(savedir + 'SV_forecast(SH=%d)_Nature_extended.svg'%(SH))
Fig.show()

# Plot the impact of forecast accuracy on the revenue using synthetic forecast 

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
from datetime import timedelta, time
import package.storage_analysis as sa
import package.get_NEM_data as gd
from calendar import monthrange
import package.optimisation as op
import package.sql_manager as sm
import plotly.graph_objects as go
import plotly.express as px
sep = ['\\','/'][sys.platform=='linux']

DB='SV_daily_synthetic_forecast_%s.db'
db=DB%('SA')

data = sm.get_data('N/A','N/A',db)
data.forecast_std =(data.forecast_std*100).astype(int) 

Fig = go.Figure()
colors = {'40':'red',
          '60':'green',
          '90':'blue'}

for RTE in [40,60,90]:
    Data = data[(data.date_time==2018)&
         (data.rte==RTE)].sort_values('forecast_std')
    DATA = Data.groupby(['forecast_std']).mean().obj_actual
    Fig.add_trace(go.Scatter(x= DATA.index,
                             y=DATA.values/DATA.values.max(),
                             error_y=dict(type='data',
                                          array=Data.groupby(['forecast_std']).std().obj_actual/DATA.values.max(),
                                          visible=True),
                             mode='markers',
                             marker =dict(color = colors[str(RTE)] ),
                             name='%d%%'%(RTE)))
    
    fit = np.polyfit(DATA.index,DATA.values/DATA.values.max(),3).tolist()
    curve = np.poly1d(fit)
    X = np.arange(0,56,1)
    Fig.add_trace(go.Scatter(x= X,
                             y=curve(X),
                             mode='lines',
                             line=dict(color = colors[str(RTE)]),
                             showlegend=False
                            ))

Fig.update_xaxes({'title':'Forecast inaccuracy [%]',
                  'dtick': 10,
                  'ticks':'outside',
#                   'tickangle':30,
                  'showgrid':True,
#                   'gridwidth':1,
                  'gridcolor':'lightgray',
                  'linecolor':'black',
                  'mirror':True,
                  'showline':True})
Fig.update_yaxes({'title':'Normalised net revenue',
                  'ticks':'outside',
#                   'dtick': 0.2,
                  'showgrid':True,
#                   'gridwidth':1,
                  'gridcolor':'lightgray',
                  'linecolor':'black',
                  'mirror':True,
                  'showline':True})

Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='v',
                  legend=dict(x=0.2, y=0.1,
                              title='     RTE',
                              xanchor='center',
                              yanchor='bottom'),
                  width=450, height=300,
                  margin=dict(l=10, r=10, t=20, b=10),
                  )
savedir = paperdir + 'Pictures%sForecast comparison%s'%(sep,sep) 
# Fig.write_image(savedir + 'SV_synthetic_forecast(SH=%d)_Nature.svg'%(10))
Fig.show()



# the error of forecast in NEM

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
import package.get_NEM_data as gd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import timedelta, time

Fig = make_subplots(
#                     subplot_titles=['RTE=40%', 'RTE=90%'],
#                     shared_yaxes=True,
                    rows=1, cols=1, horizontal_spacing=0.05)

Year = 2018
state = 'SA'
price = gd.load_rrp_cal(Year, state)
price[price>300]=300
price[price<-300]=-300

forecast = gd.load_rrp_forecast_monthly(state,Year,1)
forecast = forecast[(forecast.generated.dt.time==time(0,0))&
                    (forecast.generated.dt.date==forecast.date_time.dt.date)]
forecast=forecast.set_index('date_time').price
forecast[forecast>300]=300
forecast[forecast<-300]=-300


points = ((forecast - price[forecast.index]))
MAE = abs(points).sum()/len(forecast)/price[forecast.index].mean()



Fig.add_trace(go.Histogram(x=points,
#                                nbinsx=100,
                           xbins=dict(
                                  start=-300,
                                  end=300,
                                  size=10), # 4 months bin size
                           autobinx=False,
                           histnorm='probability',
                           marker=dict(color='lightcoral', line=dict(width=0.75, color='black')),
                           showlegend=False),row=1,col=1
                           )

# Fig.add_annotation(x=150, y=0.2,
#                    text='<b>MAE=%d%%</b>'%(MAE*100),
#                    showarrow=False,
#                    xref="x", yref="y",
#                    bordercolor='black',
#                    bgcolor='white',
#                    row=1,
#                    col=1)
# Fig.add_annotation(x=-150, y=0.2,
#                    text='<b>%d</b>'%(Year),
#                    showarrow=False,
#                    xref="x", yref="y",
#                    bgcolor='white',
#                    row=1,
#                    col=1)



Fig.update_xaxes({'title':'Forecast error [USD/MWh]',
                  'range':[-300,310],
                  'dtick': 100,
                  'ticks':'outside',
#                   'tickangle':30,
                  'showgrid':True,
#                   'gridwidth':1,
                  'gridcolor':'lightgray',
                  'linecolor':'black',
                  'mirror':True,
                  'showline':True},row=1,col=1)
Fig.update_yaxes({'title':'Frequency',
                  'range':[0,0.25],
                  'ticks':'outside',
#                   'dtick': 0.2,
                  'showgrid':True,
#                   'gridwidth':1,
                  'gridcolor':'lightgray',
                  'linecolor':'black',
                  'mirror':True,
                  'showline':True},row=1,col=1)

Fig.update_layout(plot_bgcolor='white',
                  width=450, height=300,
                  margin=dict(l=0, r=15, t=30, b=0),
                  )

paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1"


savedir = paperdir + '\Pictures\Forecast comparison'
Fig.write_image(savedir + '\AEMOs_accuracy_histogram_Nature_SA.svg')


Fig.show()


# Create the forecast inaccuracy of AEMO for various years

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import package.sql_manager as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sep = ['\\','/'][sys.platform=='linux']

state = 'SA'
Fig = make_subplots(
#                     subplot_titles=['RTE=40%', 'RTE=90%'],
#                     shared_yaxes=True,
                    rows=2, cols=2, horizontal_spacing=0.05)

for y,Year in enumerate(np.arange(2016,2020,1)):

    price = gd.load_rrp_cal(Year, state)
    price[price>300]=300
    price[price<-300]=-300

    forecast = gd.load_rrp_forecast_monthly(state,Year,1)
    forecast = forecast[(forecast.generated.dt.time==time(0,0))&
                        (forecast.generated.dt.date==forecast.date_time.dt.date)]
    forecast=forecast.set_index('date_time').price
    forecast[forecast>300]=300
    forecast[forecast<-300]=-300


    points = ((forecast - price[forecast.index]))
    MAE = abs(points).sum()/len(forecast)/price[forecast.index].mean()



    Fig.add_trace(go.Histogram(x=points,
#                                nbinsx=100,
                               xbins=dict(
                                      start=-300,
                                      end=300,
                                      size=10), # 4 months bin size
                               autobinx=False,
                               histnorm='probability',
                               marker=dict(color='lightcoral', line=dict(width=0.75, color='black')),
                               showlegend=False),row=y//2+1,col=np.mod(y,2)+1
                               )
    
    Fig.add_annotation(x=150, y=0.3,
                       text='<b>MAE=%d%%</b>'%(MAE*100),
                       showarrow=False,
                       xref="x", yref="y",
                       bordercolor='black',
                       bgcolor='white',
                       row=y//2+1,
                       col=np.mod(y,2)+1)
    Fig.add_annotation(x=-150, y=0.3,
                       text='<b>%d</b>'%(Year),
                       showarrow=False,
                       xref="x", yref="y",
                       bgcolor='white',
                       row=y//2+1,
                       col=np.mod(y,2)+1)



    Fig.update_xaxes({'title':['','Forecast error [$/MWh]'][Year in [2018,2019]],
                      'range':[-300,310],
                      'dtick': 100,
                      'ticks':'outside',
    #                   'tickangle':30,
                      'showgrid':True,
    #                   'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True},row=y//2+1,col=np.mod(y,2)+1)
    Fig.update_yaxes({'title':['','Frequency'][np.mod(y,2)==0],
                      'range':[0,0.4],
                      'ticks':'outside',
    #                   'dtick': 0.2,
                      'showgrid':True,
    #                   'gridwidth':1,
                      'gridcolor':'lightgray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True},row=y//2+1,col=np.mod(y,2)+1)

Fig.update_layout(plot_bgcolor='white',
                  width=900, height=600,
                  margin=dict(l=0, r=4, t=0, b=0),
                  )
paperdir = r"C:\Users\z3337922\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sForecast comparison%s'%(sep,sep) 
# Fig.write_image(savedir + 'AEMOs_accuracy_histogram_Nature_SA_2016_2019.svg')
Fig.show()



# Plot synthetic forecast against actual price

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)   
import numpy as np
import package.get_NEM_data as gd
from datetime import time
import pandas as pd
import plotly.graph_objects as go

Year = 2018
state = 'SA'

rrp_yearly = gd.load_rrp_cal(Year, state).append(
             gd.load_rrp_cal(Year+1, state)[0:480])

forecast = pd.DataFrame(rrp_yearly)
forecast['MEAN'] = 1
forecast['STD'] = 0.5
forecast['random'] = np.random.normal(loc=forecast.MEAN, scale=forecast.STD)
forecast['price'] = forecast.spot_price*forecast.random


Fig = go.Figure()

Fig.add_trace(go.Scatter(x=forecast.index, y=forecast.spot_price,line_shape='hv',
               name='actual'))

Fig.add_trace(go.Scatter(x=forecast.index, y=forecast.price,
              line_shape='hv',name='forecast'))

Fig.update_layout(
    xaxis=dict(rangeselector=dict(buttons=list([
                dict(count=1,label="1d",step="day",stepmode="backward"),
                dict(count=7,label="1w",step="day",stepmode="backward"),
                dict(count=1,label="1m",step="month",stepmode="backward"),
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(step="all")])
                                 ),
        rangeslider=dict(visible=True),type="date"),
   yaxis=dict(title='price [A$/MWh]',
             range=[0,300])
)

Fig.show()

# Plot the 300-AUD cap contract data

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)

import numpy as np
import pandas as pd
from projdirs import paperdir
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import package.get_NEM_data as gd

sep = ['\\','/'][sys.platform=='linux']
paperdir = r"C:\Users\Ahmad Mojiri\OneDrive - Australian National University\Publications\paper_1\\"
savedir = paperdir + 'Pictures%sCap price effect%s'%(sep,sep) 


data = pd.read_excel(r'C:\NextCloud\Thermal Battery Research\data\NEM\ASX Energy Caps data.xlsx',
                     usecols=[0,5,8,9,11,12,14])
# Start plotting
Region = 'SA'
data_SA = data[data.Location==Region].drop('Location',axis=1)
data_SA = data_SA[data_SA.Date.dt.year<2023]
data_SA['price_year'] = data_SA.Date.dt.year
data_SA['Price'] = data_SA[['Price','price_year']].apply(
                    lambda x: gd.convert_curr(*x ),axis=1)
data_SA = data_SA.drop('price_year', axis=1)



# define a custom function to calculate the volume weighted average prices for resampling the data
def vwa(X):
    if X['Volume [MWh]'].sum()==0:
        return 0
    else:
        return((X['Price']*X['Volume [MWh]']).sum()/(X['Volume [MWh]'].sum()))


cap_prices = pd.DataFrame()

cap_prices['Price VWA'] = data_SA.groupby(['Year','Period']).resample('M', on='Date').apply(vwa)

cap_volume = pd.DataFrame()
cap_volume = data_SA.groupby(['Year','Period']).resample('M', on='Date').sum() 

cap_data = cap_prices.copy()
cap_data['Volume [MWh]'] = cap_volume['Volume [MWh]']
cap_data = cap_data.reset_index()

    
years = np.arange(2010,2023,1)
    
for Q in range(1,5,1):
    Fig = make_subplots(rows=1, cols=1)
    cap_data_Q = cap_data[(data.Period=='Q%d'%(Q))].copy()
    size_max=cap_data_Q['Volume [MWh]'].max()
    price_max = cap_data_Q['Price VWA'].max()
    
    for Year in years:
        if (Year==2010) & (Q==1):
            continue

        DATA = cap_data_Q[cap_data_Q.Year==Year]
        Size = DATA['Volume [MWh]']
        DATA = DATA[DATA['Volume [MWh]']>0]
        Fig.add_trace(go.Scatter(x = DATA.Date,
                                 y = DATA['Price VWA'],
                                 mode='markers',
                                 name=str(Year)+'-'+'Q%d'%(Q),
                                 showlegend =True,
                                 marker = dict(size = DATA['Volume [MWh]']/5000)
                                ))
        
    Fig.add_trace(go.Scatter(x = [2011],
                             y = [price_max],
                             mode='markers',
                             name='Volume',
                             showlegend =False,
                             marker_size = 100000/5000,
                             marker_color = 'gray'
                              ))
    
    Fig.add_annotation(x=datetime.datetime(2012,6,1), y=price_max,
                       text='100,000 MWh',
                       showarrow=False,
                       xref="x", yref="y",
                       bgcolor='white')
    
    Fig.update_xaxes({'ticks':'outside',
                      'showgrid':True,
                      'gridcolor':'gray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True,
                     }, row=1, col=1)

    Fig.update_yaxes({'title':'Price [USD/MWh]',
                      'ticks':'outside',
                      'showgrid':True,
                      'gridcolor':'gray',
                      'linecolor':'black',
                      'mirror':True,
                      'showline':True}, row=1, col=1)

    Fig.update_layout(width=700, height=350,
                      plot_bgcolor='white',
                      legend_itemsizing='constant',
                      margin=dict(l=10, r=10, t=20, b=10))


#     Fig.write_image(savedir + 'cap_prices_SA_2010_2022_Q%d.png'%(Q))
Fig.show()


# plot the impact of SH on SV

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)

curr_path = os.path.realpath('..')

if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)

print("Working directory is now %s" %curr_path)

import numpy as np
import pandas as pd
from projdirs import datadir, resultsdir
import package.sql_manager as sm
import package.get_NEM_data as gd
from plotly.subplots import make_subplots
import plotly.graph_objects as go


PJM = ['PJM-RTO', 'MID-ATL/APS', 'AECO', 'BGE', 'DPL', 'JCPL', 'METED',
             'PECO', 'PEPCO', 'PPL', 'PENELEC', 'PSEG', 'RECO', 'APS', 'AEP',
             'COMED', 'DAY', 'DOM']
NEM = ['SA','NSW','VIC','QLD','TAS']

regions = ['SA', 'BGE']
titles = ['SA', 'PJM']

RTEs = np.arange(90,20,-60)
SHours = np.arange(18,0,-1)
loss = 0
cap = 14500
Colors = ['orangered','green','mediumseagreen','orange','deepskyblue','mediumslateblue','violet','gold']

years = [2015,2020]
Fig = make_subplots(rows=1, cols=2,
                    subplot_titles=titles,
                    horizontal_spacing=0.05,
                    vertical_spacing=0.1)
for R,Region in enumerate(regions):
    if Region in NEM:
        db = 'storage_value_PF_simple_%s.db'%(Region)
        reg_type = 'state'
    if Region in PJM:
        db = 'storage_value_PF_simple_PJM.db'
        reg_type = 'zone'
        
    table = 'storage_value'
    cols = [x[0] for x in sm.list_columns(db,table)]
    data = sm.get_data(cols,table, db)
    for i,year in enumerate(years):
        for r,RTE in enumerate(RTEs):
            SV = data[(data['date_time']==year) &
                      (data[reg_type]==Region)&
                      (data['rte']==RTE) &
                      (data['loss']==loss)&
                      (data['Pin_max']==50)&
                      (data['Pout_max']==50)&
                      (data['sh']<19)]
            if reg_type=='state':
                SV=SV[SV['cap']==cap]
                SV['SV']= SV[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)

            SV = SV.sort_values('sh', axis=0)
            SV_values = SV[SV.sh>=1].set_index('sh')['SV']
#             pdb.set_trace()
            SV_norm = SV_values/SV_values.loc[1]
            
            Fig.add_trace(go.Scatter(x=SV_norm.index,
                                     y=SV_norm,
                                     mode='lines',
                                     line = dict(width=2,
                                                 dash=['solid','dash'][i==0],
                                                 color=Colors[r]),
                                     name='%d%%'%(RTEs[r]),
        #                                  marker=dict(color=Colors[r], line_width=1, size=7),
                                     legendgroup='%d'%(RTE),
                                     showlegend = False),
                                     row=1, col=R+1)
            j=5+5*i+3*int(RTE/50)
            
            Fig.add_annotation(x=SV_norm.index[j],y=SV_norm.iloc[j], text='%d(%d%%)'%(year,RTE),
                               font=dict(size=12, color=Colors[r]),
                               row=1,col=R+1, bgcolor="white")

for i in [1,2]:
    for j in [1,2]:
        Fig.update_xaxes({'title':['','SH [Hours]'][i==1],
                          'tickmode': 'linear',
                          'tick0': 0,
                          'dtick': 2,
                          'showticklabels':[False,True][i==1],
                          'ticks':'outside',
                          'tickangle':0,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        Fig.update_yaxes({'title':[r'ANR/ANR<sub>SH=1<sub>',''][j==2],
                          'ticks':'outside',
                          'range':[1,2.4],
                          'nticks':14,
                          'showgrid':True,
                          'gridwidth':1,
                          'gridcolor':'lightgray',
                          'linecolor':'black',
                          'mirror':True,
                          'showline':True}, row=i, col=j)
        
Fig.update_layout(plot_bgcolor='white',
                  legend_orientation='h',
                  legend=dict(x=0.5, y=1.02,xanchor='center',yanchor='bottom'),
                  legend_title_text = 'RTE',
                  width=900, height=350,
                  margin=dict(l=10, r=10, t=20, b=10),
                  barmode='overlay')

paperdir = r"C:\\Users\\Ahmad Mojiri\\OneDrive - Australian National University\Publications\\paper_1\\"
savedir = paperdir + 'Pictures\Storage value vs SH_line chart\\' 


Fig.write_image(savedir + 'SV_SH_%s_%d_%d_extended.svg'%("-".join(regions),years[0],years[-1]))
Fig.show()

In [ ]:
import sys, os
print("Operating system: %s" %sys.platform)
if sys.platform == 'win32':
    curr_path = 'c:\\Nextcloud\\Thermal Battery Research\\modelling\\python'
elif sys.platform == 'linux':
    curr_path = '/home/jeff/cloud/documents/work/ANU/Thermal Battery Research/modelling/python'
else: print("What operating system are you running?! I've never even heard of %s" %sys.platform)
if curr_path not in sys.path:
    sys.path.append(curr_path)
    print('Path added! \n')
os.chdir(curr_path)
print("Working directory is now %s" %curr_path)
import numpy as np
import numpy_financial as npf
import pandas as pd
from projdirs import datadir, figdir, resultsdir, paperdir
import matplotlib.pyplot as plt
import matplotlib as mpl
import package.sql_manager as sm
import pdb
import matplotlib.gridspec as gridspec
import package.get_NEM_data as gd
from matplotlib import container, cm
import itertools
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline
plt.rcParams['text.usetex'] = True
sep = ['\\','/'][sys.platform=='linux']
def er(x,y):
    return(y-av([x,y]))

CPE_range = np.array(list(
                        itertools.chain(np.arange(1,10,1),
                                        np.arange(10,110,2),#2
                                        np.arange(110,900,5),#5
                                        np.arange(900,1500,10)
                                       )
                         ) )# $/kWh

                           
CPP_range = np.array(list( 
                        itertools.chain(np.arange(10,200,2),#2
                                        np.arange(200,6000,20) )#20
                        ) ) # $/kW


IRR = np.zeros( (len(CPP_range), len(CPE_range)) )
RTEs = [40]
Pout_max = 50 #MWe
Pin_max = 50 #MWe
SH = 8
Year = 2015
life = 20

excel_file = 'C:/Users/Ahmad Mojiri/OneDrive - Australian National University/Publications/paper_1/Data/CPE_CPP.xlsx'
systems = pd.read_excel(excel_file, 'Summary',header=0, usecols='A:O',nrows=37)

fig, Axes = plt.subplots(figsize=(8,6),nrows=1, ncols=1, gridspec_kw = {'wspace':0.05, 'hspace':0.1})
Ax=np.ravel(Axes)
font_size = 16


Region = 'SA'
db='SV_NEM_cap_contract.db'
data = sm.get_data('N/A','N/A',db)

db = 'storage_value_PF_with_Eout_SA.db'
e_out = sm.get_data('N/A','N/A',db)

SV = data[(data.state==Region)&
          (data.sh==SH)&
          (data.loss==0)&
          (data.cap==14500)&
          (data.Pin_max==Pin_max)&
          (data.Pout_max==Pout_max)&
          (data.date_time>Year)].sort_values('date_time')#['SV']

E_out = e_out[(e_out.state==Region)&
              (e_out.loss==0)&
              (e_out.cap==14500)&
              (e_out.sh==SH)&
              (e_out.Pin_max==Pin_max)&
              (e_out.Pout_max==Pout_max)&
              (e_out.date_time>Year)].sort_values('date_time').copy()



# Convert the NEM SV to usd
SV['SV_total']= data[['SV_total','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
SV['SV']= data[['SV','date_time']].apply(lambda x: gd.convert_curr(*x ),axis=1)
# convert to 2015 USD
SV['SV'] = SV.apply(lambda x: gd.time_value(x['SV'],x['date_time'],2015,0.02 ),axis=1)
SV['SV_total'] = SV.apply(lambda x: gd.time_value(x['SV_total'],x['date_time'],2015,0.02 ),axis=1)


for r,RTE in enumerate(RTEs):
    i=int(np.divide(r,2))+1
    j=int(np.mod(r,2))+1
    SV_rte = SV[SV['rte']==RTE]['SV_total']
    SV_mean = SV_rte.mean()
    E_rte = E_out[E_out.rte==RTE]['Eout']
    E_OUT = E_rte.mean()
    
    for cpe,CPE in enumerate(CPE_range):
        for cpp,CPP in enumerate(CPP_range):
            CAPEX = Pout_max*CPP*1e3+CPE*Pout_max*SH*1e3
            cashflows = np.insert(SV_mean*np.ones(life),0,-CAPEX)
            IRR[cpp,cpe] = round(npf.irr(cashflows),8)

    X, Y = np.meshgrid(CPE_range, CPP_range)
    IRR[IRR>1.0]=1.01
    IRR[IRR<0]=-0.0001
    
#     L = [0,0.03,0.06,0.08,0.12,0.18,0.3, 0.4, 0.5, 0.6, 0.9, 1]
    L=[0, 0.03, 0.06, 0.08, 0.12, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8]
    
    CMAP = cm.get_cmap('CMRmap')#BuGn  Wistia GnBu  YlGn  Blues  GnBu gnuplot jet
    colors = CMAP(np.linspace(0.65, 1, num=len(L)+1))
    colors_rep = [x for x in colors.tolist() for i in range(2)]
    
    L_rep = [x for x in L for i in range(2)]
    L_rep[0]=0.0005
    L_rep[1]=0.0005
    L_rep[-2]=0.8
    L_rep[-1]=0.8
    L_rep = [0]+L_rep+[1]
    
    CMAP = LinearSegmentedColormap.from_list(
                'mycmap', [x for x in zip(L_rep,colors_rep)])
    
    im = Ax[r].contourf(X,Y, IRR, 1000, cmap=CMAP)
    Ax[r].set_yscale('log')
    Ax[r].set_xscale('log')
    
    im2 = Ax[r].contour(X,Y, IRR, colors='k',levels=L)
    clabels = Ax[r].clabel(im2, inline_spacing=5, colors = 'k', fmt = '%2.2f', fontsize=font_size-2)

    Ax[r].grid(which='both', axis='both')
    Ax[r].set_ylabel(r'CPP [$\$$/kW]', fontsize=font_size)
    Ax[r].set_xlabel(r'CPE [$\$$/kWh]', fontsize=font_size)
    
    Systems = systems[(systems['Eff_low']*100<=RTE) & (systems['Eff_high']*100>=RTE)]
    for i in [19,36]:
        CPE_mean = Systems.loc[i,'CPE_mean']
        CPP_mean = Systems.loc[i,'CPP_mean']
        CAPEX = Pout_max*(CPP_mean)*1e3+(CPE_mean)*Pout_max*SH*1e3
        FOM = Systems.loc[i,'FOM']*Pout_max*1e3
        VOM = Systems.loc[i,'VOM']*E_OUT*1e3
        
        cashflows = np.insert((SV_mean-FOM-VOM)*np.ones(life),0,-CAPEX)
        IRR_median = ( round(npf.irr(cashflows),4)  )
        
        FOM_npv = npf.npv(IRR_median,np.insert(FOM*np.ones(life),0,0))
        VOM_npv = npf.npv(IRR_median,np.insert(VOM*np.ones(life),0,0))
        
        CPP_fom = FOM_npv/(Pout_max*1e3)
        CPE_vom = VOM_npv/(Pout_max*SH*1e3)
        
       
        Ax[r].errorbar(CPE_mean+CPE_vom, CPP_mean+CPP_fom,
                       xerr=np.array([[Systems.loc[i,'CPE_err-'],Systems.loc[i,'CPE_err+']]]).T, 
                       yerr=np.array([[Systems.loc[i,'CPP_err-'],Systems.loc[i,'CPP_err+']]]).T, 
                       marker=Systems.loc[i,'Marker'],
                       color=Systems.loc[i,'color'],
                       mec='b',ms=10, label = Systems.loc[i,'Type_ref']
                      ,ecolor='b', capsize=5,linestyle = 'None')
       
    Ax[r].text(0.87,0.85,'RTE=%d\%% \n SH=%d'%(RTE,SH),
              fontsize=font_size, transform=Ax[r].transAxes,
               ha='center', bbox=dict(facecolor='w', edgecolor='k'), zorder=10)

    Ax[r].set_ylim([CPP_range.min(),CPP_range.max()])
    Ax[r].set_xlim([CPE_range.min(),CPE_range.max()])
    handles, labels = Ax[r].get_legend_handles_labels()
    handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
    lg = Ax[r].legend(handles, labels,
                      loc='lower left', ncol=1,
                      fontsize=font_size-4,
                      edgecolor='k',
                      handletextpad=0.5,
                      framealpha=0.8)

import matplotlib.ticker as tick
# cbar.ax.yaxis.set_major_formatter(tick.FormatStrFormatter('%.1f'))
plt.rc('ytick', labelsize=font_size)
plt.rc('xtick', labelsize=font_size)


savedir = r'C:\\Users\\Ahmad Mojiri\\OneDrive - Australian National University\\Presentations\\'

# plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d)_with_cap_ with retrofit_Nature.png'%(Year+1,SH),dpi=600,  format='png',bbox_inches = "tight")
# plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d)_with_cap_ with retrofit_Nature.svg'%(Year+1,SH),bbox_inches = "tight")
plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d).svg'%(Year+1,SH),bbox_inches = "tight")
plt.savefig(savedir + 'IRR_CPP_CPE_%d-2019 (SH=%d).png'%(Year+1,SH),bbox_inches = "tight")

plt.show()